In [1]:
from IPython.display import JSON

# Imports main tools:
from trulens_eval import Feedback, Tru, LiteLLM, Huggingface, TruBasicApp
tru = Tru(database_file="trulens_db.sqlite")
tru.reset_database()

🦑 Tru initialized with db url sqlite:///trulens_db.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


/tmp/ipykernel_95555/1704003846.py:5: DeprecationWarning: `database_file` is deprecated, use `database_url` instead as in `database_url='sqlite:///filename'.
  tru = Tru(database_file="trulens_db.sqlite")


In [2]:
from google.cloud import aiplatform
from vertexai.preview.generative_models import GenerativeModel, Part
import os

In [3]:
aiplatform.init(
            project = "ultra-heading-407815",
            location= "us-central1",
        )

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'cloud_credentials.json'

In [5]:
model = GenerativeModel("gemini-pro-vision")

In [6]:
import base64
image_path = "image.jpeg"
img = open(image_path, "rb").read()
img_bytes = Part.from_data(base64.b64decode(base64.encodebytes(img)), mime_type="image/jpeg")

In [7]:
responses = model.generate_content(["explain this image", img_bytes])

In [8]:
responses.text

' This is a photo of a puppy. The breed is an Entlebucher Mountain Dog.'

In [9]:
prompt = "explain this image"

def llm_standalone(prompt, img=img_bytes):
    return model.generate_content([
        prompt
    ]+ [img_bytes, img_bytes]).text

In [10]:
llm_standalone("explain this 2 images, tell me whether it is same", img_bytes)

' The two images are the same.'

In [11]:
# Initialize Huggingface-based feedback function collection class:
hugs = Huggingface()

# Define a sentiment feedback function using HuggingFace.
f_sentiment = Feedback(hugs.positive_sentiment).on_output()

✅ In positive_sentiment, input text will be set to __record__.main_output or `Select.RecordOutput` .


In [12]:
# creating a recorder app with the feedback functions

tru_app_recorder = TruBasicApp(llm_standalone, app_id="Sentiment bot", feedbacks=[f_sentiment])

In [13]:
with tru_app_recorder as records:
    tru_app_recorder.app("Explain the image")

In [14]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [15]:
# tru.stop_dashboard()